In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.2 MB/s eta 0:00:00


In [3]:
pip install pandas numpy pywavelets scikit-learn plotly

In [4]:
# 📦 Importations
import numpy as np
import pandas as pd
import pywt
import pickle
import plotly.graph_objs as go
import plotly.express as px
from sklearn.metrics import mean_squared_error

# 📂 Recharger les fichiers générés
loaded = np.load("/content/signal_compresse.npz", allow_pickle=True)
compressed_arr = loaded["compressed_arr"]
wavelet = str(loaded["wavelet"])
original_len = int(loaded["original_len"])

with open("/content/coeff_slices.pkl", "rb") as f:
    coeff_slices = pickle.load(f)

# 🔁 Reconstruction des coefficients
coeffs = pywt.array_to_coeffs(compressed_arr, coeff_slices, output_format='wavedec')

# 🔧 Reconstruction du signal
reconstructed_signal = pywt.waverec(coeffs, wavelet)[:original_len]

# 📉 Charger le signal original pour comparaison
file_path = "/content/drive/MyDrive/P2M/sfmconnect.disaggregationP2M.csv"
df = pd.read_csv(file_path)

# 🧼 Nettoyage
df["equipments[0].equipment.name"] = df["equipments[0].equipment.name"].astype(str).str.strip()
df_projector = df[df["equipments[0].equipment.name"] == "retro-projecteur"]
signal_original = df_projector["equipments[0].consumption.Active_energy"].dropna().values
timestamps = pd.to_datetime(df_projector["date"], unit='ms')

# ✨ Évaluation
rmse = np.sqrt(mean_squared_error(signal_original[:original_len], reconstructed_signal))
print(f"✅ Signal reconstruit avec ondelette '{wavelet}' — RMSE : {rmse:.6f}")

# 📊 Affichage interactif Plotly
fig = go.Figure()

# Signal original
fig.add_trace(go.Scatter(
    x=timestamps[:original_len],
    y=signal_original[:original_len],
    mode='lines',
    name="Signal original",
    line=dict(color='black', width=2)
))

# Signal reconstruit
fig.add_trace(go.Scatter(
    x=timestamps[:original_len],
    y=reconstructed_signal,
    mode='lines',
    name=f"Reconstruit ({wavelet})",
    line=dict(dash='dash', color=px.colors.qualitative.Set1[1])
))

# Mise en forme du graphique
fig.update_layout(
    title="🔁 Signal original vs. reconstruit (Plotly)",
    xaxis_title="Temps",
    yaxis_title="Énergie active (kWh)",
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=600
)

fig.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/P2M/sfmconnect.disaggregationP2M.csv'